## ニューラルネットワークモデル比較
## (sklearn vs pytorch vs 手動)
---

### モデル構造

- 入力層：特徴量数に依存（今回のデータセットでは8次元）  
- 隠れ層：16ユニット、活性化関数はシグモイド関数  
- 出力層：1ユニット、シグモイド活性化により確率を出力   

---



入力 X に対し、隠れ層・出力層の出力は以下のように計算：

$$
Z^{(1)} = X W_1 + b_1
$$

$$
A^{(1)} = \sigma(Z^{(1)})
$$

$$
Z^{(2)} = A^{(1)} W_2 + b_2
$$

$$
A^{(2)} = \sigma(Z^{(2)})
$$
 は入力特徴量の次元数

ニューロンの出力はシグモイド関数で計算：

$$
\sigma(z) = \frac{1}{1 + e^{-z}}
$$



---

### 損失関数

出力 $A^{(2)}$ と正解ラベル $y$ ,サンプル数$m$を用いて損失は

$$
L = -\frac{1}{m} \sum_{i=1}^{m} \left[ y_i \log A_i^{(2)} + (1 - y_i) \log (1 - A_i^{(2)}) \right]
$$



---


### パラメータ更新（勾配降下法）

学習率 $\alpha$ を用いてパラメータを更新：

$$
W_1 \leftarrow W_1 - \alpha \nabla_{W_1}
$$

$$
b_1 \leftarrow b_1 - \alpha \nabla_{b_1}
$$

$$
W_2 \leftarrow W_2 - \alpha \nabla_{W_2}
$$

$$
b_2 \leftarrow b_2 - \alpha \nabla_{b_2}
$$

---



In [ ]:
#ライブラリのインポート
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
import time
import torch
import torch.nn as nn
import torch.optim as optim

In [22]:
# データ読み込みと前処理

data = pd.read_csv("data/diabetes.csv")
X = data.drop("Outcome", axis=1).values
y = data["Outcome"].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_norm = scaler.fit_transform(X_train)
X_test_norm = scaler.transform(X_test)

In [ ]:
#手動ニューラルネットワーク（1層）実装

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def sigmoid_derivative(z):
    return sigmoid(z) * (1 - sigmoid(z))

# ネットワーク構造：入力 → 中間層(16) → 出力(1)
input_dim = X_train_norm.shape[1]
hidden_dim = 16
output_dim = 1
alpha = 0.1
epochs = 1000
m = X_train_norm.shape[0]

np.random.seed(42)
W1 = np.random.randn(input_dim, hidden_dim)
b1 = np.zeros(hidden_dim)
W2 = np.random.randn(hidden_dim, output_dim)
b2 = np.zeros(output_dim)

start_time = time.time()
for epoch in range(epochs):
    # フォワード
    Z1 = np.dot(X_train_norm, W1) + b1
    A1 = sigmoid(Z1)
    Z2 = np.dot(A1, W2) + b2
    A2 = sigmoid(Z2).flatten()
    
    # ロス（クロスエントロピー）
    epsilon = 1e-7
    loss = -np.mean(y_train * np.log(A2 + epsilon) + (1 - y_train) * np.log(1 - A2 + epsilon))
    
    # バックプロパゲーション
    dZ2 = A2 - y_train
    dW2 = np.dot(A1.T, dZ2[:, np.newaxis]) / m
    db2 = np.mean(dZ2, axis=0)
    dA1 = np.dot(dZ2[:, np.newaxis], W2.T)
    dZ1 = dA1 * sigmoid_derivative(Z1)
    dW1 = np.dot(X_train_norm.T, dZ1) / m
    db1 = np.mean(dZ1, axis=0)
    
    # パラメータ更新
    W1 -= alpha * dW1
    b1 -= alpha * db1
    W2 -= alpha * dW2
    b2 -= alpha * db2

    if epoch % 100 == 0:
        print(f"[manual NN] Epoch {epoch}: Loss = {loss:.4f}")
end_time = time.time()
print(f"[manual NN] 実行時間: {end_time - start_time:.4f} 秒")

# 評価
Z1_test = np.dot(X_test_norm, W1) + b1
A1_test = sigmoid(Z1_test)
Z2_test = np.dot(A1_test, W2) + b2
A2_test = sigmoid(Z2_test).flatten()
y_pred_manual = (A2_test >= 0.5).astype(int)
acc_manual = accuracy_score(y_test, y_pred_manual)
print(f"Accuracy (manual NN): {acc_manual:.4f}")



[manual NN] Epoch 0: Loss = 0.8859
[manual NN] Epoch 100: Loss = 0.5520
[manual NN] Epoch 200: Loss = 0.4970
[manual NN] Epoch 300: Loss = 0.4774
[manual NN] Epoch 400: Loss = 0.4676
[manual NN] Epoch 500: Loss = 0.4616
[manual NN] Epoch 600: Loss = 0.4575
[manual NN] Epoch 700: Loss = 0.4544
[manual NN] Epoch 800: Loss = 0.4518
[manual NN] Epoch 900: Loss = 0.4496
[manual NN] 実行時間: 1.1176 秒
Accuracy (manual NN): 0.7597


In [ ]:
# PyTorchによる実装
class SimpleNN(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.Sigmoid(),
            nn.Linear(16, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x)

# 入力次元
input_dim = X_train_norm.shape[1]
epochs = 1000


torch.manual_seed(42)
model = SimpleNN(input_dim)
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# テンソル変換
X_train_tensor = torch.tensor(X_train_norm, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.reshape(-1, 1), dtype=torch.float32)

# 学習
start_time = time.time()
for epoch in range(epochs):
    y_pred = model(X_train_tensor)
    loss = criterion(y_pred, y_train_tensor)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if epoch % 100 == 0:
        print(f"[PyTorch NN] Epoch {epoch}: Loss = {loss.item():.4f}")
end_time = time.time()
print(f"[PyTorch NN] 実行時間: {end_time - start_time:.4f} 秒")

# 評価
X_test_tensor = torch.tensor(X_test_norm, dtype=torch.float32)
y_pred_torch = model(X_test_tensor).detach().numpy().flatten()
y_pred_torch_bin = (y_pred_torch >= 0.5).astype(int)
acc_torch = accuracy_score(y_test, y_pred_torch_bin)
print(f"Accuracy (PyTorch NN): {acc_torch:.4f}")


[PyTorch NN] Epoch 0: Loss = 0.7865
[PyTorch NN] Epoch 100: Loss = 0.5866
[PyTorch NN] Epoch 200: Loss = 0.5327
[PyTorch NN] Epoch 300: Loss = 0.4975
[PyTorch NN] Epoch 400: Loss = 0.4791
[PyTorch NN] Epoch 500: Loss = 0.4698
[PyTorch NN] Epoch 600: Loss = 0.4650
[PyTorch NN] Epoch 700: Loss = 0.4623
[PyTorch NN] Epoch 800: Loss = 0.4607
[PyTorch NN] Epoch 900: Loss = 0.4596
[PyTorch NN] 実行時間: 0.8222 秒
Accuracy (PyTorch NN): 0.7532


In [ ]:
# sklearnによる実装
# モデル定義
model = MLPClassifier(hidden_layer_sizes=(16,),
                      activation='logistic',  # sigmoid活性化
                      solver='sgd',
                      learning_rate_init=0.1,
                      max_iter=1000,
                      random_state=42,
                      batch_size=X_train_norm.shape[0],  # バッチ学習に近づける
                      verbose=True)  # 学習ログを表示

start_time = time.time()
model.fit(X_train_norm, y_train)
end_time = time.time()
# 予測
y_pred = model.predict(X_test_norm)
#　精度
acc = accuracy_score(y_test, y_pred)

print(f"Test accuracy: {acc:.4f}")
print(f"Training time: {end_time - start_time:.4f} seconds")

Iteration 1, loss = 0.67893666
Iteration 2, loss = 0.67330685
Iteration 3, loss = 0.66765829
Iteration 4, loss = 0.66298742
Iteration 5, loss = 0.65940345
Iteration 6, loss = 0.65646312
Iteration 7, loss = 0.65358626
Iteration 8, loss = 0.65034892
Iteration 9, loss = 0.64659622
Iteration 10, loss = 0.64241006
Iteration 11, loss = 0.63800049
Iteration 12, loss = 0.63358660
Iteration 13, loss = 0.62931285
Iteration 14, loss = 0.62522003
Iteration 15, loss = 0.62126439
Iteration 16, loss = 0.61736233
Iteration 17, loss = 0.61343454
Iteration 18, loss = 0.60943248
Iteration 19, loss = 0.60534271
Iteration 20, loss = 0.60117583
Iteration 21, loss = 0.59695032
Iteration 22, loss = 0.59268006
Iteration 23, loss = 0.58836930
Iteration 24, loss = 0.58401442
Iteration 25, loss = 0.57960922
Iteration 26, loss = 0.57515047
Iteration 27, loss = 0.57064137
Iteration 28, loss = 0.56609229
Iteration 29, loss = 0.56151951
Iteration 30, loss = 0.55694289
Iteration 31, loss = 0.55238369
Iteration 32, los